In [1]:
import os
import pandas as pd
import torch

from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    Trainer, 
    TrainingArguments, 
    BitsAndBytesConfig, 
    Trainer, pipeline
)
from peft import LoraConfig
from datasets import Dataset
from langchain.prompts.prompt import PromptTemplate

from trl import SFTTrainer
from peft import PeftModel


/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
new_model = "./finetune_models/llama2"

In [3]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map={"":0}
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.47s/it]


In [6]:
# fname = "finetune_data_llama2b.csv"
fname = os.getenv("HOME") + "/pwdata/mazharul/llama2-model-train-pws/train.txt"
df_data = []
with open(fname, 'r') as fin:
    for line in fin:
        pw1, pw_others = line.strip().split('\t', 1)
        df_data.append([pw1, pw_others])
        
df_data = pd.DataFrame(df_data, columns=["pw_selected", "pw_others"])
df_data

,pw_selected,pw_others
0,jkljkljkl,jaolang
1,3_@myspace.com:ilovematt!,pimpin&gt:_@myspace.com:1blunt
2,peppone,zitto
3,4026292,Feniks100890
4,5420sanderhas5420,lunch15467
...,...,...
102935634,zzzzzzzzzz,6j4nghr
102935635,guns789321,lynn124
102935636,123456a,12345
102935637,1234567,123456


In [20]:
instruction = "You are given a list of passwords chosen by an user.  I will ask you chose another likely password for the user based on the given list of passwords."
question = "What is the another likely password of the user"

In [8]:
prompt_template = PromptTemplate(
    input_variables=["instruction", "passwords_chosen", "question", "answer"], template="<s>[INST] <<SYS>>{instruction}. Here is the list  (separated by '\t') {pw_others}<</SYS>>{question}[/INST]{answer}</s>"
)

prompt_data = []
for i, row in df_data.iterrows():
  prompt_data.append(prompt_template.format(instruction=instruction, pw_others = row.pw_others, question = question, answer=row.pw_selected))

dataset = Dataset.from_dict({"inputs": prompt_data})
# dataset.to_json("./finetune_data_password.json")

In [5]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='./finetune_data_password.json', split='train')

In [6]:
seed = 42
import random 
random.seed(seed)

shuffled_dataset = dataset.shuffle(seed=seed)


sampled_dataset = shuffled_dataset.select(range(1000))

In [7]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.config.use_cache = False
model.config.pretraining_tp = 1

In [8]:
peft_params = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [9]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True
)

In [10]:
trainer = SFTTrainer(
    model=model,
    train_dataset=sampled_dataset,
    peft_config=peft_params,
    dataset_text_field="inputs",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 1000/1000 [00:00<00:00, 14583.30 examples/s]
/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/accelerate/accelerator.py:427: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [11]:
trainer.train()

/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/accelerate/accelerator.py:1301: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  new_forward = torch.cuda.amp.autocast(dtype=torch.float16)(model_forward_func)
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
1,5.237200
2,4.960800
3,4.911400
4,4.501100
5,4.207800
6,4.010500
7,3.497900
8,2.899100
9,2.789100
10,2.701200


/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant

TrainOutput(global_step=1250, training_loss=0.5930917937994004, metrics={'train_runtime': 653.2863, 'train_samples_per_second': 7.654, 'train_steps_per_second': 1.913, 'total_flos': 1.0459691857870848e+16, 'train_loss': 0.5930917937994004, 'epoch': 5.0})

In [12]:

trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('./finetune_models/llama2/tokenizer_config.json',
 './finetune_models/llama2/special_tokens_map.json',
 './finetune_models/llama2/tokenizer.model',
 './finetune_models/llama2/added_tokens.json',
 './finetune_models/llama2/tokenizer.json')

In [14]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
new_model = "./llama2_password"

In [15]:
# 1. Load based model
base_model = AutoModelForCausalLM.from_pretrained(model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.38s/it]


In [16]:
# 2. Load new fine-tuned model. Then merge this two model
finetuned_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = finetuned_model.merge_and_unload()

/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


In [17]:
# 3. Load tokenizer,
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [18]:
# 4. Define pipeline
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer, max_length=2000)

/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/rahul/anaconda3/envs/omniquant/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


In [21]:
prompt_template = PromptTemplate(
    input_variables=["instruction", "passwords_chosen", "question", "answer"], template="<s>[INST] <<SYS>>{instruction}. Here is the list  (separated by '\t') {pw_others}<</SYS>>{question}[/INST]"
)
pw_others = "mauser"
input = prompt_template.format(instruction=instruction, pw_others = pw_others, question = question)
input

"<s>[INST] <<SYS>>You are given a list of passwords chosen by an user.  I will ask you chose another likely password for the user based on the given list of passwords.. Here is the list  (separated by '\t') mauser<</SYS>>What is the another likely password of the user[/INST]"

In [22]:
# 5. Run prompt and pipeline
result = pipe(input)

In [23]:
print(result[0]['generated_text'].split("/INST")[1][1:])

2210120519980000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [24]:
result

[{'generated_text': "<s>[INST] <<SYS>>You are given a list of passwords chosen by an user.  I will ask you chose another likely password for the user based on the given list of passwords.. Here is the list  (separated by '\t') mauser<</SYS>>What is the another likely password of the user[/INST]221012051998000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000